<a href="https://colab.research.google.com/github/pythonkvs/seminars/blob/main/%D0%90%D1%81%D0%B8%D0%BD%D1%85%D1%80%D0%BE%D0%BD%D0%BD%D0%BE%D1%81%D1%82%D1%8C_28_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()


<h3 id="Обработка-запросов-в-один-поток">Обработка запросов в один поток<a class="anchor-link" href="#Обработка-запросов-в-один-поток"></a></h3><ul>
<li>Подходы для организации выполнения кода в одном потоке</li>
<li>Модуль select</li>
<li>Неблокирующий ввод/вывод</li>
<li>Python фреймворки</li>
</ul>



<h3 id="Модуль-select">Модуль select<a class="anchor-link" href="#Модуль-select"></a></h3><ul>
<li>Модуль select используется для организации неблокирующего ввода/вывода.</li>
<li>Существуют несколько механизмов опроса файловых дескрипторов:<ul>
<li>select.select(...)</li>
<li>select.poll(...)</li>
<li>select.epoll(...)</li>
<li>select.kqueue(...)</li>
<li>...</li>
</ul>
</li>
</ul>


In [ ]:
# Неблокирующий ввод/вывод, обучающий пример

import socket
import select

sock = socket.socket()
sock.bind(("", 10001))
sock.listen()

# как обработать запросы для conn1 и conn2
# одновременно без потоков?
conn1, addr = sock.accept()
conn2, addr = sock.accept()

conn1.setblocking(0)
conn2.setblocking(0)
    
epoll = select.epoll()
epoll.register(conn1.fileno(), select.EPOLLIN | select.EPOLLOUT)
epoll.register(conn2.fileno(), select.EPOLLIN | select.EPOLLOUT)

conn_map = {
    conn1.fileno(): conn1,
    conn2.fileno(): conn2,
}

In [ ]:
# Неблокирующий ввод/вывод, обучающий пример
# Цикл обработки событий в epoll

while True:
    events = epoll.poll(1)
    
    for fileno, event in events:
        if event & select.EPOLLIN:
            # обработка чтения из сокета
            data=conn_map[fileno].recv(1024)
            print(data.decode("utf8"))
        elif event & select.EPOLLOUT:
            # обработка записи в сокет
            conn_map[fileno].send("pong".encode("utf8"))

В современных ОС Linux используют epoll.
При помощи вызова epoll.poll можно получить файловые дескрипторы, готовые для чтения или записи.
Такой код иногда называют асинхронным программированием, 
или мультиплексирование ввода/вывода.
Пример сделан с целью обучения для понимания того, как использовать неблокирующий ввод/вывод.


<h3 id="Неблокирующий-ввод/вывод">Неблокирующий ввод/вывод<a class="anchor-link" href="#Неблокирующий-ввод/вывод"></a></h3><ul>
<li>Код уже не выглядит слишком простым (хотя в нем нет создания потоков или процессов)<ul>
<li>Нет обработки закрытия сокетов</li>
<li>Отсутствует обработка новых входящих соединений</li>
</ul>
</li>
<li>Если код будет решать настоящие задачи, то увеличится кол-во операторов if или callback-ов</li>
<li>Не тратим память на создание процессов</li>
<li>Нет накладных расходов на создание потоков и их синхронизацию</li>
<li>Нет проблем с GIL</li>
</ul>



<h3 id="Фреймворки">Фреймворки<a class="anchor-link" href="#Фреймворки"></a></h3><ul>
<li>Twisted, callback api<ul>
<li><a href="https://twistedmatrix.com">https://twistedmatrix.com</a></li>
</ul>
</li>
<li>Gevent, greenlet, stackless python<ul>
<li><a href="http://www.gevent.org/">http://www.gevent.org/</a></li>
</ul>
</li>
<li>Tornado, generators api<ul>
<li><a href="http://www.tornadoweb.org">http://www.tornadoweb.org</a></li>
</ul>
</li>
<li>Asyncio, mainstream<ul>
<li><a href="https://docs.python.org/3/library/asyncio.html">https://docs.python.org/3/library/asyncio.html</a></li>
</ul>
</li>
</ul>



<h3 id="Итераторы-и-генераторы,-в-чем-разница?">Итераторы и генераторы, в чем разница?<a class="anchor-link" href="#Итераторы-и-генераторы,-в-чем-разница?"></a></h3><ul>
<li>Как устроены итераторы и генераторы</li>
<li>Сходства и различия</li>
</ul>


In [3]:
# Итераторы

class MyRangeIterator:
    def __init__(self, top):
        self.top = top
        self.current = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.current >= self.top:
            raise StopIteration
        
        current = self.current
        self.current += 1
        return current

In [4]:
counter = MyRangeIterator(3)
counter

In [5]:
for it in counter:
    print(it)

0
1
2


In [6]:
# Генераторы

def my_range_generator(top):
    current = 0
    while current < top:
        yield current 
        current += 1

In [7]:
counter = my_range_generator(3)
counter

<generator object my_range_generator at 0x7f417fe6f150>

In [8]:
for it in counter:
    print(it)

0
1
2



<h3 id="Итераторы-и-генераторы,-подводим-итоги">Итераторы и генераторы, подводим итоги<a class="anchor-link" href="#Итераторы-и-генераторы,-подводим-итоги"></a></h3><ul>
<li>Рассмотрели примеры работы итераторов и генераторов</li>
<li>Они решают одну и туже задачу - генерация последовательностей</li>
<li>Итератор хранит значения для следующей итерации в self</li>
<li>Генератор использует локальные переменные</li>
<li>В генераторы заложены большие возможности для написания concurrency кода</li>
</ul>



<h3 id="Генераторы-и-сопрограммы">Генераторы и сопрограммы<a class="anchor-link" href="#Генераторы-и-сопрограммы"></a></h3><ul>
<li>Как устроены сопрограммы?</li>
<li>Отличие между генераторами и сопрограммами</li>
<li>Как работает yield from?</li>
<li>Примеры работы сопрограмм</li>
</ul>


In [9]:
# Сопрограммы (корутины)

def grep(pattern):
    print("start grep")
    while True:
        line = yield
        if pattern in line:
            print(line)

In [10]:
g = grep("python")
next(g) # g.send(None)

start grep


In [11]:
g.send("golang is better?")
g.send("python is simple!")

python is simple!


In [12]:
# Сопрограммы, вызов метода close()

def grep(pattern):
    print("start grep")
    try:
        while True:
            line = yield
            if pattern in line:
                print(line)
    except GeneratorExit:
        print("stop grep")

In [13]:
g = grep("python")
next(g)  # g.send(None)

start grep


In [14]:
g.send("python is the best!")

python is the best!


In [15]:
g.close()

stop grep


In [16]:
# Сопрограммы, генерация исключений

def grep(pattern):
    print("start grep")
    try:
        while True:
            line = yield
            if pattern in line:
                print(line)
    except GeneratorExit:
        print("stop grep")

In [17]:
g = grep("python")
next(g) # g.send(None)
g.send("python is the best!")
g.throw(RuntimeError, "something wrong")

start grep
python is the best!


RuntimeError: ignored

In [18]:
# Вызовы сопрограмм, PEP 380

def grep(pattern):
    print("start grep")
    while True:
        line = yield
        if pattern in line:
            print(line)

def grep_python_coroutine():
    g = grep("python")
    next(g)
    g.send("python is the best!")
    g.close()

In [19]:
g = grep_python_coroutine()  # is g coroutine?

start grep
python is the best!


In [20]:
g

In [21]:
# Сопрограммы, yield from PEP 0380

def grep(pattern):
    print("start grep")
    while True:
        line = yield
        if pattern in line:
            print(line)

def grep_python_coroutine():
    g = grep("python")
    yield from g

In [22]:
g = grep_python_coroutine()  # is g coroutine?
g

<generator object grep_python_coroutine at 0x7f41830d6f50>

In [23]:
 g.send(None)

start grep


In [ ]:
g.send("python wow!")

In [24]:
# PEP 380, генераторы

def chain(x_iterable, y_iterable):
    yield from x_iterable
    yield from y_iterable

def the_same_chain(x_iterable, y_iterable):
    for x in x_iterable:
        yield x
    
    for y in y_iterable:
        yield y
    
a = [1, 2, 3]
b = (4, 5)
for x in chain(a, b):
    print(x)

1
2
3
4
5



<h3 id="Генераторы-и-сопрограммы,-подводим-итоги">Генераторы и сопрограммы, подводим итоги<a class="anchor-link" href="#Генераторы-и-сопрограммы,-подводим-итоги"></a></h3><ul>
<li>Как устроены генераторы и сопрограммы</li>
<li>Несмотря на некоторую схожесть, у генератора и корутины два важных отличия:<ul>
<li>Генераторы "производят" значения (yield item)</li>
<li>Корутины "потребляют" значения (item = yield)</li>
</ul>
</li>
<li>Корутина может иметь два состояния: suspended и resumed</li>
<li>yield приостанавливает корутину</li>
<li>send() возобновляет работу корутины</li>
<li>close() завершает выполнение</li>
<li>yield from используется для делегирования вызова генератора</li>
</ul>



<h3 id="Первые-шаги-с-asyncio">Первые шаги с asyncio<a class="anchor-link" href="#Первые-шаги-с-asyncio"></a></h3><ul>
<li>Введение в asyncio</li>
<li>Примеры выполнения asyncio кода</li>
</ul>



<h3 id="Первые-шаги-с-asyncio">Первые шаги с asyncio<a class="anchor-link" href="#Первые-шаги-с-asyncio"></a></h3><p>Фреймворк asyncio - это:</p>
<ul>
<li>часть Python3</li>
<li>неблокирующий ввод/вывод</li>
<li>сервисы с тысячами соединений одновременно</li>
<li>в основе лежат генераторы и корутины</li>
<li>линейный код, отсутствие callbacks!</li>
</ul>


In [25]:
# asyncio, Hello World

import asyncio

@asyncio.coroutine
def hello_world():
    while True:
        print("Hello World!")
        yield from asyncio.sleep(1.0)

loop = asyncio.get_event_loop()
loop.run_until_complete(hello_world())
loop.close()

Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!


KeyboardInterrupt: ignored

Hello World!


In [26]:
# asyncio, async def / await; PEP 492 Python3.5

import asyncio

async def hello_world():
    while True:
        print("Hello World!")
        await asyncio.sleep(1.0)

In [27]:
loop = asyncio.get_event_loop()
loop.run_until_complete(hello_world())
loop.close()

Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!


KeyboardInterrupt: ignored

Hello World!


In [ ]:
# asyncio, tcp сервер

import asyncio

async def handle_echo(reader, writer):
    data = await reader.read(1024)
    message = data.decode()
    addr = writer.get_extra_info("peername")
    print("received %r from %r" % (message, addr))
    writer.close()
    
loop = asyncio.get_event_loop()
coro = asyncio.start_server(handle_echo, "127.0.0.1", 10001, loop=loop)
server = loop.run_until_complete(coro)
try:
    loop.run_forever()
except KeyboardInterrupt:
    pass

server.close()
loop.run_until_complete(server.wait_closed())
loop.close()

In [ ]:
# asyncio, tcp клиент

import asyncio

async def tcp_echo_client(message, loop):
    reader, writer = await asyncio.open_connection("127.0.0.1", 10001, loop=loop)

    print("send: %r" % message)
    writer.write(message.encode())
    writer.close()

loop = asyncio.get_event_loop()
message = "hello World!"
loop.run_until_complete(tcp_echo_client(message, loop))
loop.close()


<h3 id="Выполнение-кода-в-asyncio">Выполнение кода в asyncio<a class="anchor-link" href="#Выполнение-кода-в-asyncio"></a></h3><ul>
<li>asyncio.Future</li>
<li>asyncio.Task</li>
<li>loop.run_in_executor</li>
<li>библиотеки для работы с asyncio</li>
</ul>


In [4]:
### asyncio.Future, аналог concurrent.futures.Future

import asyncio

async def slow_operation(future):
    await asyncio.sleep(1)
    future.set_result("Future is done!")

In [5]:
loop = asyncio.get_event_loop()
future = asyncio.Future()
asyncio.ensure_future(slow_operation(future))

loop.run_until_complete(future)
print(future.result())

Future is done!


In [ ]:
loop.close()

In [7]:
### asyncio.Task, запуск нескольких корутин

import asyncio

async def sleep_task(num):
    for i in range(5):
        print(f"process task: {num} iter: {i}")
        await asyncio.sleep(1)
    
    return num

# ensure_future or create_task
loop = asyncio.get_event_loop()

task_list = [loop.create_task(sleep_task(i)) for i in range(2)]
loop.run_until_complete(asyncio.wait(task_list))

loop.run_until_complete(loop.create_task(sleep_task(3)))
loop.run_until_complete(asyncio.gather(
    sleep_task(10),
    sleep_task(20),
    ))

process task: 0 iter: 0
process task: 1 iter: 0
process task: 0 iter: 1
process task: 1 iter: 1
process task: 0 iter: 2
process task: 1 iter: 2
process task: 0 iter: 3
process task: 1 iter: 3
process task: 0 iter: 4
process task: 1 iter: 4
process task: 3 iter: 0
process task: 3 iter: 1
process task: 3 iter: 2
process task: 3 iter: 3
process task: 3 iter: 4
process task: 10 iter: 0
process task: 20 iter: 0
process task: 10 iter: 1
process task: 20 iter: 1
process task: 10 iter: 2
process task: 20 iter: 2
process task: 10 iter: 3
process task: 20 iter: 3
process task: 10 iter: 4
process task: 20 iter: 4


[10, 20]

In [8]:
# loop.run_in_executor, запуск в отдельном потоке

import asyncio
from urllib.request import urlopen

# a synchronous function
def sync_get_url(url):
   return urlopen(url).read()

async def load_url(url, loop=None):
    future = loop.run_in_executor(None, sync_get_url, url)
    response = await future
    print(len(response))

loop = asyncio.get_event_loop()
loop.run_until_complete(load_url("https://google.com", loop=loop))

14865



<h3 id="Библиотеки-asyncio">Библиотеки asyncio<a class="anchor-link" href="#Библиотеки-asyncio"></a></h3><ul>
<li><a href="https://github.com/aio-libs">https://github.com/aio-libs</a></li>
<li>aiohttp<ul>
<li><a href="https://github.com/aio-libs/aiohttp">https://github.com/aio-libs/aiohttp</a></li>
</ul>
</li>
<li>aiomysql<ul>
<li><a href="https://github.com/aio-libs/aiomysql">https://github.com/aio-libs/aiomysql</a></li>
</ul>
</li>
<li>aiomcache<ul>
<li><a href="https://github.com/aio-libs/aiomcache">https://github.com/aio-libs/aiomcache</a></li>
</ul>
</li>
<li><a href="https://docs.python.org/3/library/asyncio.html">https://docs.python.org/3/library/asyncio.html</a></li>
</ul>
